In [81]:
# ! pip install transformers

In [110]:

import os
import re
import numpy as np
import json

import requests
from bs4 import BeautifulSoup

import torch
import transformers

In [9]:
def get_metadata():
    with open('arxiv/arxiv-metadata-oai-snapshot.json', 'r') as f:
        for line in f:
            yield line

In [12]:
relevant_cats = {
'cond-mat.dis-nn': 'Disordered Systems and Neural Networks',
'cs.AI': 'Artificial Intelligence',
'cs.CC': 'Computational Complexity',
'cs.CE': 'Computational Engineering, Finance, and Science',
'cs.CL': 'Computation and Language',
'cs.CR': 'Cryptography and Security',
'cs.CV': 'Computer Vision and Pattern Recognition',
'cs.DB': 'Databases',
'cs.DC': 'Distributed, Parallel, and Cluster Computing',
'cs.DL': 'Digital Libraries',
'cs.DM': 'Discrete Mathematics',
'cs.DS': 'Data Structures and Algorithms',
'cs.ET': 'Emerging Technologies',
'cs.FL': 'Formal Languages and Automata Theory',
'cs.GL': 'General Literature',
'cs.GR': 'Graphics',
'cs.GT': 'Computer Science and Game Theory',
'cs.IR': 'Information Retrieval',
'cs.IT': 'Information Theory',
'cs.LG': 'Machine Learning',
'cs.LO': 'Logic in Computer Science',
'cs.MA': 'Multiagent Systems',
'cs.MM': 'Multimedia',
'cs.MS': 'Mathematical Software',
'cs.NA': 'Numerical Analysis',
'cs.NE': 'Neural and Evolutionary Computing',
'cs.NI': 'Networking and Internet Architecture',
'cs.OH': 'Other Computer Science',
'cs.OS': 'Operating Systems',
'cs.PF': 'Performance',
'cs.PL': 'Programming Languages',
'cs.RO': 'Robotics',
'cs.SC': 'Symbolic Computation',
'cs.SD': 'Sound',
'cs.SE': 'Software Engineering',
'cs.SI': 'Social and Information Networks',
'cs.SY': 'Systems and Control',
'econ.EM': 'Econometrics',
'eess.AS': 'Audio and Speech Processing',
'eess.IV': 'Image and Video Processing',
'eess.SP': 'Signal Processing',
'math.AC': 'Commutative Algebra',
'math.AG': 'Algebraic Geometry',
'math.AP': 'Analysis of PDEs',
'math.AT': 'Algebraic Topology',
'math.CA': 'Classical Analysis and ODEs',
'math.CO': 'Combinatorics',
'math.CT': 'Category Theory',
'math.CV': 'Complex Variables',
'math.DG': 'Differential Geometry',
'math.DS': 'Dynamical Systems',
'math.FA': 'Functional Analysis',
'math.GM': 'General Mathematics',
'math.GN': 'General Topology',
'math.GR': 'Group Theory',
'math.GT': 'Geometric Topology',
'math.HO': 'History and Overview',
'math.IT': 'Information Theory',
'math.KT': 'K-Theory and Homology',
'math.LO': 'Logic',
'math.MG': 'Metric Geometry',
'math.MP': 'Mathematical Physics',
'math.NA': 'Numerical Analysis',
'math.NT': 'Number Theory',
'math.OA': 'Operator Algebras',
'math.OC': 'Optimization and Control',
'math.PR': 'Probability',
'math.QA': 'Quantum Algebra',
'math.RA': 'Rings and Algebras',
'math.RT': 'Representation Theory',
'math.SG': 'Symplectic Geometry',
'math.SP': 'Spectral Theory',
'math.ST': 'Statistics Theory',
'math-ph': 'Mathematical Physics',
'q-fin.CP': 'Computational Finance',
'q-fin.MF': 'Mathematical Finance',
'q-fin.RM': 'Risk Management',
'q-fin.ST': 'Statistical Finance',
'q-fin.TR': 'Trading and Market Microstructure',
'stat.AP': 'Applications',
'stat.CO': 'Computation',
'stat.ME': 'Methodology',
'stat.ML': 'Machine Learning',
'stat.OT': 'Other Statistics',
'stat.TH': 'Statistics Theory'}

In [50]:
metadata = get_metadata()
i = 0
with open('arxiv/arxiv-metadata-oai-snapshot.json', 'r') as f:
        for line in f:
            json_line = json.loads(line)
            print(json_line)
            break
# for paper in metadata:
#     if i % 343 == 0:
    
#         for k, v in json.loads(paper).items():
#             print(f'{k}: {v}')
#         break
#     i += 1

{'id': '0704.0001', 'submitter': 'Pavel Nadolsky', 'authors': "C. Bal\\'azs, E. L. Berger, P. M. Nadolsky, C.-P. Yuan", 'title': 'Calculation of prompt diphoton production cross sections at Tevatron and\n  LHC energies', 'comments': '37 pages, 15 figures; published version', 'journal-ref': 'Phys.Rev.D76:013009,2007', 'doi': '10.1103/PhysRevD.76.013009', 'abstract': '  A fully differential calculation in perturbative quantum chromodynamics is\npresented for the production of massive photon pairs at hadron colliders. All\nnext-to-leading order perturbative contributions from quark-antiquark,\ngluon-(anti)quark, and gluon-gluon subprocesses are included, as well as\nall-orders resummation of initial-state gluon radiation valid at\nnext-to-next-to-leading logarithmic accuracy. The region of phase space is\nspecified in which the calculation is most reliable. Good agreement is\ndemonstrated with data from the Fermilab Tevatron, and predictions are made for\nmore detailed tests with CDF and 

In [42]:
metadata = get_metadata()

In [74]:
categories

['math.NT', 'math.AG']

In [89]:

i = 0
j = 0
first_thousand = []
relevant_list = list(relevant_cats.keys())
with open('arxiv/ml_papers_snapshot_2.json', 'w') as write_file:
    with open('arxiv/arxiv-metadata-oai-snapshot.json', 'r') as f:
        for line in f:
            json_paper = json.loads(line)
            categories = json_paper['categories'][0].split(" ")

            if not any([x in relevant_list for x in categories]):
                continue
            try:
                date = re.search("^(\d+)\..*", json_paper['id']).group(1)
            except:
                date = None

            if int(date) < 1500:
                continue

            json_paper['categories'] = categories
            json_paper['date'] = int(date)
            json_paper['year'] = int(f'20{date[:2]}')

            json.dump(json_paper, write_file)
            write_file.write('\n')



            i += 1
            if i == 100000:
                break


In [97]:
i = 0

papers = []
header = '"id"|year"|"tags"|"text"\n'
with open('arxiv/arxiv_data.csv', 'w') as writefile:
    writefile.write(header)
    
    with open('arxiv/ml_papers_snapshot_2.json') as f:
        for line in f:
            json_paper = json.loads(line)
            id_ = json_paper['id']
            categories = json_paper['categories']
            year = json_paper['year']
            abstract = json_paper['abstract']
            
            write_line = f'"{id_}"|"{year}"|"{categories}"|"{abstract}"'
            writefile.write(write_line)
            writefile.write("\n")
            

            i += 1
            if i % 10000 == 0:
                print(i)


10000
20000
30000
40000
50000
60000
70000
80000
90000
100000


In [98]:
json_paper

{'id': '1611.04432',
 'submitter': '- Departement Mathematiques Orsay',
 'authors': 'Jean-Pierre Kahane (LMO)',
 'title': "Conditions for Beurling's integers to have a density",
 'comments': 'in French',
 'journal-ref': None,
 'doi': None,
 'abstract': "  In 1997 H.G.Diamond gave a condition on Beurling's generalized prime numbers\nin order that the corresponding generalized integers have a density. We give a\nnew proof of this condition (Theorem 1) and a proof that it is not necessary\n(Theorem 2 and Examples). However, it is very near to be necessary (Theorem 3).\nBoth proofs of Theorems 1 and 2 rely on Fourier analysis, mainly the Wiener\nalgebra, and partly on probability methods.\n",
 'report-no': None,
 'categories': ['math.NT'],
 'versions': ['v1'],
 'date': 1611,
 'year': 2016}

In [95]:
import csv
with open('arxiv/arxiv_data.csv', newline='') as f:
    reader = csv.reader(f, delimiter='|', quoting=csv.QUOTE_ALL)
    for row in reader:
        print(row)

['id', 'year"', 'tags', 'text']
['1501.00001', '2015', "['cs.IT', 'cs.NI', 'math.IT']", '  Distinction of OFDM signals from single carrier signals is highly important\nfor adaptive receiver algorithms and signal identification applications. OFDM\nsignals exhibit Gaussian characteristics in time domain and fourth order\ncumulants of Gaussian distributed signals vanish in contrary to the cumulants\nof other signals. Thus fourth order cumulants can be utilized for OFDM signal\nidentification. In this paper, first, formulations of the estimates of the\nfourth order cumulants for OFDM signals are provided. Then it is shown these\nestimates are affected significantly from the wireless channel impairments,\nfrequency offset, phase offset and sampling mismatch. To overcome these\nproblems, a general chi-square constant false alarm rate Gaussianity test which\nemploys estimates of cumulants and their covariances is adapted to the specific\ncase of wireless OFDM signals. Estimation of the covari

In [113]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')

In [102]:
tokenizer

In [121]:
['introduction to algorithms', 'the elements of statistical learning']
test_token = tokenizer.encode(['introduction to algorithms', 'the elements of statistical learning'])

print(test_token)
print(tokenizer.convert_ids_to_tokens(test_token))

[101, 100, 100, 102]
['[CLS]', '[UNK]', '[UNK]', '[SEP]']


In [117]:
test_tensor = torch.LongTensor(test_token)
print(test_tensor)


model = transformers.BertModel.from_pretrained('bert-base-uncased', output_hidden_states=True)
model = model.to(device)
test_tensor = test_tensor.to(device)

model.eval()
print(model)

tensor([  101,  4955,  2000, 13792,   102])
BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropou

In [118]:
print(test_tensor.size())
test_tensor = test_tensor.unsqueeze(0)
print(test_tensor.size())

print(type(test_tensor))
with torch.no_grad():
    out = model(input_ids=test_tensor)


print(type(out))
print(len(out))

hidden_states = out[2]
print(len(hidden_states))

torch.Size([5])
torch.Size([1, 5])
<class 'torch.Tensor'>
<class 'tuple'>
3
13


In [119]:
sentence_embedding = torch.mean(hidden_states[-1], dim=1).squeeze()
print(sentence_embedding)
print(sentence_embedding.size())

tensor([-2.6710e-01,  2.6857e-02, -7.1127e-01, -1.6682e-01,  7.5849e-02,
         1.0340e-01,  1.4030e-01,  1.9636e-01, -2.6274e-02, -6.5242e-02,
         6.3598e-02, -2.1960e-01, -3.4613e-01, -3.5831e-01, -3.7734e-01,
         5.8638e-03, -4.2269e-02,  3.9169e-02, -2.6189e-01,  2.9446e-01,
         2.4032e-01,  5.2831e-01, -1.1056e-01,  4.6630e-01,  1.5466e-01,
         3.0498e-02,  3.6937e-02,  3.4433e-01, -3.4441e-01,  1.5501e-01,
         6.6546e-02,  4.1077e-01, -2.1893e-01, -9.6625e-02, -2.3164e-01,
         2.1648e-01,  3.1286e-01,  6.0638e-02, -5.5950e-01,  5.1150e-01,
        -5.9491e-01,  3.4276e-01,  4.0101e-01,  2.5867e-01, -1.4939e-02,
        -7.0332e-01, -3.8815e-01,  5.0209e-02, -1.6225e-01, -2.3893e-01,
        -2.4715e-01,  1.1856e-01,  5.7324e-01,  6.2366e-02,  2.1231e-01,
         6.6461e-01,  1.0568e-01, -6.1718e-01,  1.7590e-01, -2.6281e-01,
         9.9906e-02,  2.4703e-01,  9.7846e-02, -2.5034e-01,  6.3563e-01,
         4.3127e-01, -1.2051e-01,  5.2409e-01, -4.8

In [120]:
# get last four layers
last_four_layers = [hidden_states[i] for i in (-1, -2, -3, -4)]
# cast layers to a tuple and concatenate over the last dimension
cat_hidden_states = torch.cat(tuple(last_four_layers), dim=-1)
print(cat_hidden_states.size())

# take the mean of the concatenated vector over the token dimension
cat_sentence_embedding = torch.mean(cat_hidden_states, dim=1).squeeze()
print(cat_sentence_embedding)
print(cat_sentence_embedding.size())

torch.Size([1, 5, 3072])
tensor([-0.2671,  0.0269, -0.7113,  ..., -0.0484, -0.1982,  0.3941])
torch.Size([3072])


In [ ]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
max_len -= tokenizer.num_special_tokens_to_add()

with open(dataset, "rt") as f_p:
    for line in f_p:
        line = line.rstrip()

        if not line:
            print(line)
            subword_len_counter = 0
            continue

        token = line.split()[0]

        current_subwords_len = len(tokenizer.tokenize(token))

        # Token contains strange control characters like \x96 or \x95
        # Just filter out the complete line
        if current_subwords_len == 0:
            continue

        if (subword_len_counter + current_subwords_len) > max_len:
            print("")
            print(line)
            subword_len_counter = current_subwords_len
            continue

        subword_len_counter += current_subwords_len

        print(line)